In [ ]:
pip install vaderSentiment

Note the webscraping is based on: https://www.youtube.com/watch?v=CJAdCLZaISw&t=850s

In [ ]:
pip install psaw 

In [ ]:
#import packages
import pandas as pd
import numpy as np
from psaw import PushshiftAPI
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# Select number of days to look back
number_days = 365

In [ ]:
# Get wsb feed from pushshift.io
api = PushshiftAPI()
start_time = int((datetime.datetime.today() - datetime.timedelta(days=number_days)).timestamp()) 
end_time = int(datetime.datetime.today().timestamp())
submissions = list(api.search_submissions(after=start_time,
                                          before=end_time,
                                     subreddit='wallstreetbets',
                                     filter=['url','author', 'title', 'subreddit', 'date']))



In [ ]:
#check 
submissions

In [ ]:
#check 2
type(submissions)

In [ ]:
# Filter for cashtags
df_list = []
for submission in submissions:
    words = submission.title.split()
    cashtags = list(set(filter(lambda word: word.lower().startswith('$'), words)))
    if len(cashtags) > 0:
        dict_data = {"Cashtag": cashtags[0].replace('$',''),
                     "Title": submission.title,
                     "created": submission.created}
        df_list.append(dict_data)
wsb = pd.DataFrame.from_dict(df_list)

In [ ]:
#import package
import datetime as dt

In [ ]:
#function transforming timestamp
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [ ]:
_timestamp = wsb["created"].apply(get_date)

In [ ]:
wsb = wsb.assign(timestamp = _timestamp)

In [ ]:
#check
wsb.head()

In [ ]:
#rename column
wsb.rename(columns={'timestamp': 'date'}, inplace= True)

In [ ]:
#store webscraped results 
#wsb.to_csv('wsb_1_year.csv')

In [ ]:
import pandas as pd

In [ ]:
wsb = pd.read_csv (r'/Users/michellemichalowski/Downloads/wsb1.csv')

In [ ]:
wsb = wsb.dropna()

In [ ]:
#clean cashtags 1
wsb = wsb[wsb['Cashtag'].str.contains('^[A-Z]')]

In [ ]:
#clean cashtags 2
wsb = wsb[wsb['Cashtag'].str.contains('[A-Z]$')]

In [ ]:
#gather only needed columns
wsb = wsb[['Cashtag','Title','date']]

In [ ]:
wsb.head()

,Cashtag,Title,date
0,CVE,Energy Oil &amp; Exp was the sector to be in t...,2021-04-28 20:26:09
1,PLTR,$AMC AND $PLTR primed for takeoff 🚀🚀🚀,2021-04-28 20:18:57
2,BUZZ,My thoughts and explanation of $BUZZ,2021-04-28 20:11:50
3,PLTR,$AMC and $PLTR primed for takeoff,2021-04-28 20:09:10
4,BLNK,GM signed agreement with $BLNK green💚💚🌎🌎🌎🌎,2021-04-28 20:05:47


In [ ]:
#import vader 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
#define lexicon for slang and specific emojis with positive sentiment
new_words = {
'pump': 4,
'dump': -4,
'pumping': 4,
'dumping': -4,
'bailout': 4,
'bailed out': 4,
'crazy': 4,
'spike': 4,
'spiking': 4,
'printer': 4,
'print': 4,
'printing': 4,
'puts will print': -4,
'brr': 4,
'brrr': 4,
'brrrr': 4,
'brrrrr': 4,
'brrrrrr': 4,
'brrrrrrr': 4,
'brrrrrrrr': 4,
'brrrrrrrrr': 4,
'bless': 4,
'blood': -4,
'sue': -4,
'suing': -4,
'garbage': -4,
'trash': -4,
'die': -4,
'drill': -4,
'drilling': -4,
'fraud': -4,
'buy': 4,
'sell': -4,
'sell-off': -4,
'selloff': -4,
'bought': 4,
'sold': -4,
'buying': 4,
'selling': -4,
'rid': -4,
'call': 4,
'calls': 4,
'c': 4,
'p': -4,
'put': -4,
'puts': -4,
'poots': -4,
'up': 4,
'down': -4,
'gain': 4,
'gainz': 4,
'gained': 4,
'gaining': 4,
'gains': 4,
'grow': 4,
'grew': 4,
'growing': 4,
'made': 4,
'making': 4,
'win': 4,
'profit': 4,
'winning': 4,
'profiting': 4,
'lose': -4,
'losing': -4,
'loss': -4,
'lost': -4,
'$ROPE': -4,
'stonk': 4,
'stonks': 4,
'chad': 4,
'chads': 4,
'virgin': -4,
'virgins': -4,
'strong': 4,
'weak': -4,
'downturn': -4,
'green': 4,
'red': -4,
'permabull': 4,
'permabulls': 4,
'bull': 4,
'bulls': 4,
'bullish': 4,
'bulls are fucked': -4,
'bulls are fuked': -4,
'bear': -4,
'bears': -4,
'bearish': -4,
'bears are fucked': 4,
'bears are fuked': 4,
'rip call holders': -4,
'rip put holders': 4,
'leap': 4,
'leaps': 4,
'ree': -4,
'reee': -4,
'reeee': -4,
'reeeee': -4,
'reeeeee': -4,
'tendie': 4,
'tendies': 4,
'top': -4,
'bottom': 4,
'long': -4,
'short': -4,
'shorts': -4,
'shorting': -4,
'wrong': -4,
'right': 4,
'yolo': 4,
'yoloed': 4,
'yolod': 4,
'broke': -4,
'good': 4,
'bad': -4,
'better': 4,
'best': 4,
'worse': -4,
'worst': -4,
'sad': -4,
'big': 4,
'soaring': 4,
'soar': 4,
'soared': 4,
'to the moon': 4,    
'moon': 4,
'mooon': 4,
'mooooon': 4,
'mooned': 4,
'mooning': 4,
'moooning': 4,
'moooooning': 4,
'skyrocket': 4,
'skyrocketing': 4,
'crater': -4,
'cratering': -4,
'crash': -4,
'crashed': -4,
'crashing': -4,
'tank': -4,
'tanks': -4,
'ground': -4,
'floor': -4,
'roof': 4,
'tanking': -4,
'plummet': -4,
'plummeting': -4,
'plummeted': -4,
'plunge': -4,
'plunged': -4,
'plunging': -4,
'positive': 4,
'negative': -4,
'drop': -4,
'rise': 4,
'dropping': -4,
'rising': 4,
'dropped': -4,
'rose': 4,
'cheap': 4,
'expensive': -4,
'downswing': -4,
'upswing': 4,
'upwards': 4,
'downwards': -4,
'support': 4,
'resistance': -4,
'squeeze': 4,
'squeezed': 4,
'record': 4,
'rally': 4,
'rallying': 4,
'rallied': 4,
'surge': 4,
'surging': 4,
'surged': 4,
'correction': -4,
'optimism': 4,
'optimistic': 4,
'low': -4,
'lows': -4,
'high': 4,
'highs': 4,
'recession': -4,
'recessionary': -4,
'depression': -4,
'expansion': 4,
'expansionary': 4,
'run-up': 4,
'runup': 4,
'run': 4,
#add emojis   
'fire': 4,
'rocket': 4,
'diamond': 4,
'mushroom': 4,
'flash': 4,
'moon': 4,    
'open hands': 4,
'palms up together': 4,
'automobile': 4,
'astronaut': 4,
'man astronaut': 4,
'woman astronaut': 4,
'baby': 4,
'new moon': 4,
'waxing crescent moon':4, 
'first quarter moon': 4,
'waxing gibbous moon': 4,
'full moon': 4,
'waning gibbous moon': 4,
'last quarter moon': 4,
'waning crescent moon': 4,    
#is not working
'money-mouth face': 4,
'pill': 4,
'crescent moon': 4,
'raising hands': 4,
'pinched fingers': 4,    
'gorilla': 4
}

In [ ]:
#check if sentiments for emojis are captured correctly 
sia = SentimentIntensityAnalyzer()
sia.lexicon.update(new_words)
sia.polarity_scores('🚀')

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.7184}

In [ ]:
vader = SentimentIntensityAnalyzer()

In [ ]:
#update lexicon 
vader.lexicon.update(new_words)

In [ ]:
vader.polarity_scores(wsb['Title'])

{'neg': 0.126, 'neu': 0.589, 'pos': 0.285, 'compound': 1.0}

In [ ]:
#gather the score 
wsb['scores'] = wsb['Title'].apply(lambda title: vader.polarity_scores(title))

In [ ]:
#extract compound score 
wsb['compound'] = wsb['Title'].apply(lambda x:vader.polarity_scores(x)['compound'])

In [ ]:
wsb.head()

,Cashtag,Title,date,scores,compound
0,CVE,Energy Oil &amp; Exp was the sector to be in t...,2021-04-28 20:26:09,"{'neg': 0.098, 'neu': 0.669, 'pos': 0.232, 'co...",0.7783
1,PLTR,$AMC AND $PLTR primed for takeoff 🚀🚀🚀,2021-04-28 20:18:57,"{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'comp...",0.9517
2,BUZZ,My thoughts and explanation of $BUZZ,2021-04-28 20:11:50,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
3,PLTR,$AMC and $PLTR primed for takeoff,2021-04-28 20:09:10,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,BLNK,GM signed agreement with $BLNK green💚💚🌎🌎🌎🌎,2021-04-28 20:05:47,"{'neg': 0.0, 'neu': 0.376, 'pos': 0.624, 'comp...",0.9828


In [ ]:
import numpy as np

In [ ]:
wsb['comp_score'] = wsb['compound'].apply(lambda c: 'pos' if c >0.1 else('neg' if c < -0.1 else 'neut'))

In [ ]:
wsb.head()

,Cashtag,Title,date,scores,compound,comp_score
0,CVE,Energy Oil &amp; Exp was the sector to be in t...,2021-04-28 20:26:09,"{'neg': 0.098, 'neu': 0.669, 'pos': 0.232, 'co...",0.7783,pos
1,PLTR,$AMC AND $PLTR primed for takeoff 🚀🚀🚀,2021-04-28 20:18:57,"{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'comp...",0.9517,pos
2,BUZZ,My thoughts and explanation of $BUZZ,2021-04-28 20:11:50,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neut
3,PLTR,$AMC and $PLTR primed for takeoff,2021-04-28 20:09:10,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neut
4,BLNK,GM signed agreement with $BLNK green💚💚🌎🌎🌎🌎,2021-04-28 20:05:47,"{'neg': 0.0, 'neu': 0.376, 'pos': 0.624, 'comp...",0.9828,pos


In [ ]:
wsb.value_counts('comp_score')

comp_score
pos     30014
neut    18169
neg      7950
dtype: int64

In [ ]:
wsb.to_csv('wsb_final.csv')